<h2>Scrapes Christies Auction Records for Auction URLs and Information<h2>

In [1]:
#imports everything needed to scrape
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import numpy as np
import time
import random
import re

In [2]:
df = pd.read_csv ('Art_Object_URL.csv')

In [3]:
shuffled = df.sample(frac=1, random_state=7).reset_index()
shuffled.head(10)

,index,Art_URL
0,27856,https://www.christies.com/lot/lot-6202478?ldp_...
1,63471,https://www.christies.com/lot/lot-5813514?ldp_...
2,4299,https://onlineonly.christies.com/s/collector/l...
3,23452,https://www.christies.com/lot/lot-6233769?ldp_...
4,12077,https://onlineonly.christies.com/s/old-master-...
5,28560,https://www.christies.com/lot/lot-6196270?ldp_...
6,46805,https://www.christies.com/lot/lot-6059129?ldp_...
7,28400,https://onlineonly.christies.com/s/first-open-...
8,5394,https://onlineonly.christies.com/s/first-open-...
9,3154,https://onlineonly.christies.com/s/vente-en-li...


In [4]:
#given a path launches a website
Path = "/Users/ramirostorni/Documents/chromedriver"
driver = webdriver.Chrome(Path)

/var/folders/48/1t2lf7js69g6my_l4x0npdrw0000gn/T/ipykernel_28354/1960765232.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(Path)


In [5]:
def CookieCloser():
    try:
        cookie_event = WebDriverWait(driver, .05).until(
            EC.presence_of_element_located((By.ID ,"onetrust-accept-btn-handler"))
        )
        cookie_event.click()
    except:
        pass

In [6]:
def ClickEmail():
    try:
        email_event = WebDriverWait(driver, .05).until(
            EC.presence_of_element_located((By.ID ,"close_signup"))
        )
        time.sleep(.05)
        email_event.click()
    except:
        pass

In [7]:
def GetArtistName():
    try:
        Artist = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'chr-lot-header__artist-name'))
        )
    except:
        Artist = "No Name"
        return Artist
        
#     print("ARTIST NAME:", Artist.text)
    return Artist.text

In [8]:
def GetArtTitle():
    try:
        ArtTitle = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.TAG_NAME, "h1"))
        )
    except:
        ArtTitle = "No Name"
        return ArtTitle
    
    return ArtTitle.text

In [9]:
def GetArtPrice():
    try:
        Price = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'chr-lot-header__value-field'))
        )
    except:
        Price = "0"
        return Price
    
#     print("ART PRICE:", Price.text)
    return Price.text

In [10]:
def GetEstPrice():
    try:
        estemated_price = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'chr-lot-header__estimate-details'))
        )
    except:
        estemated_price = "0"
        return estemated_price
        
#     print("EST PRICE:", estemated_price.text)   
    return estemated_price.text

In [11]:
def GetAuctionTitle():
    try:
        AuctionTitle = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'chr-lot-header__breadcrumb-detail'))
        )
        
    except:
        AuctionTitle = "0"
        print("problem with Auction title")
        return AuctionTitle
        
    return AuctionTitle.text

In [12]:
def GetCloseDate():
    try:
        closeDate = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'chr-lot-timer'))
        )
    except:
        closeDate = "0"
        print("Small problem with close date")
        return closeDate
        
    return closeDate.text

In [13]:
def GetArtDetails():
    try:
        details = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'sect_0'))
        )
    except:
        details = "0"
        print("error in art details")
        return details
    
    return details.text

In [14]:
def getArtistLife(details):
    
    try:  
        BirthYear = re.search(r'(?<=b\. )\w+', details)
        DeathYear = "0"
        life = (BirthYear.group(0), DeathYear)
    except:
        
        try:
            BirthYear = re.search(r'(?<=b\.)\w+', details)
            DeathYear = "0"
            life = (BirthYear.group(0), DeathYear)
        except:
            
            try:
                BirthYear = re.search(r'(?<=\()\w+', details)
                DeathYear = re.search(r'(?<=-)\w+', details)
                life = (BirthYear.group(0), DeathYear.group(0))
            except:
                life = ("0", "0")
                
       
    return life

In [15]:
def getCreationDate(details):
    try:
        CreationDate = re.search(r'(?<=in )\d+', details)
    except:
        CreationDate = 0
        return CreationDate
    
    try:
        CreationDate.group(0)
    except:
        CreationDate = 0
        return CreationDate
    
    return CreationDate.group(0)

In [16]:
def GetDimentions(details):
    try:
        split_details = details.split("\n")
        for items in split_details:
            if " x " in items:
                dimentions = re.search(r'\([^()]*\)', items)
                return dimentions.group(0)   
    except:
        dimentions = "0"
        pass
        return dimentions

In [17]:
#Artist Info
Artist_Name = [] 
Artist_Birth = []
Artist_Death = [] 

#Art Info
Artwork_Title = [] 
Date_of_Creation = [] 
Dimetions = []

#Prices
Price_Estimate = [] 
Sold_Price = [] 
Close_Date = [] 

#Auction Info
Auction_Title = [] 




## Data Collection Loop

In [18]:
for URL in shuffled["Art_URL"]:
    
    #opens URL and Clicks cookies popup
    driver.get(URL)
    driver.maximize_window()
    CookieCloser()
    
    #Gets the art work detail paragraph from the webpage
    ArtDetails = GetArtDetails()
    
    #Gathers Artist information
    Artist = GetArtistName()
    Birth = getArtistLife(ArtDetails)[0]
    Death = getArtistLife(ArtDetails)[1]
    
    #Gathers Artwork information
    ArtTitle = GetArtTitle()
    Price = GetArtPrice()
    EstPrice = GetEstPrice()
    CreationDate = getCreationDate(ArtDetails)
    Dimentions_CM = GetDimentions(ArtDetails)
    
    #Gathers Auction Information
    CloseDate = GetCloseDate()
    AuctionTitle = GetAuctionTitle()
    
    
    #Appends all of the gathered info to lists
    Artist_Name.append(Artist)
    Artist_Birth.append(Birth)
    Artist_Death.append(Death)
    img_Url.append(Image_URL)
    Artwork_Title.append(ArtTitle)
    Date_of_Creation.append(CreationDate)
    Dimetions.append(Dimentions_CM)
    Price_Estimate.append(EstPrice)
    Sold_Price.append(Price)
    Close_Date.append(CloseDate)
    Auction_Title.append(AuctionTitle)
    

NameError: name 'img_Url' is not defined

## slightly cleans the data and packs it into a data frame

In [ ]:
Artists = []
for names in Artist_Name:
    CleanName = re.sub(r'\([^()]*\)', '', names)
    Artists.append(CleanName)

In [ ]:
Close = []
for dates in Close_Date:
    dates = dates.replace("\n", "")
    dates = dates.replace("Closed: ", "")
    Close.append(dates)

In [ ]:
df = pd.DataFrame(list(zip(Artists, Artist_Birth, Artist_Death, Artwork_Title, Date_of_Creation, Price_Estimate, Sold_Price,Dimetions , Close, Auction_Title)),
               columns =['Artist', 'Birth', 'Death', 'Art Title', 'DoC', 'Estimate', 'price','Dimentions', 'close date', 'Auction Title'])

df.head(100)

In [ ]:
#creates copy of the data so we can remove all works without dimentions
copy = df.copy()
copy.shape

In [ ]:
copy.drop(copy.index[copy['Dimentions'] == "0"], inplace=True)

In [ ]:
copy.shape

In [ ]:
# copy.to_csv("Art_Object_Info2.csv", index = False)